In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None):
    
    print("Actual value ", test_y)
    num_features=max_feat;
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=num_features, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [ ]:
def create_samples(shap_explainer, iterations, row, features, top = 10):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        for i in range(top):
            feat = importances[i]
            if feat[2] > 0:
                rel_feat.append(feat)
                
        rel_exp.append(rel_feat)
        
    return exp, rel_exp

In [8]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = True
generate_lime = True
generate_shap = True

sample_size = 3
exp_iter = 3
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

In [10]:
if generate_samples:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

                predictor = joblib.load(pipeline_path)
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                bucketer = joblib.load(bucketer_path)

                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previous results from predictions
                tn_path = os.path.join(PATH, "%s/%s_%s/instances/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/instances/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/instances/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/instances/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                instances.append(tn_list)
                instances.append(tp_list)
                instances.append(fn_list)
                instances.append(fp_list)

                #choose random instances from the event log to explain
                sample_instances = []
                
                for result_type in instances:
                    samples = []              
                    idxs = random.sample(range(0, len(result_type)), sample_size)
                    for i in idxs:
                        sample = result_type[i]
                        samples.append(sample)
                    sample_instances.append(samples)

                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1


In [19]:
if generate_lime:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #get the training data as a matrix
                trainingdata = feature_combiner.fit_transform(dt_train_bucket);
                #importance = generate_global_explanations(trainingdata,train_y, cls, feature_combiner)


                #create explainer now that can be passed later
                class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
                lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                              feature_names = feature_combiner.get_feature_names(),
                                              class_names=class_names, discretize_continuous=True)
                
                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    for instance in category:
                        instance_no += 1
                        
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                        
                        group = instance['input']

                        print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get lime explanations for instance
                        feat_pres = []

                        for iteration in list(range(exp_iter)):
                            print("Run", iteration)
                            lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, instance['actual'])

                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in lime_exp.as_list():
                                    if each in explanation[0]:
                                        #if lime_exp.as_list().index(explanation) < max_feat:
                                            presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['lime_stability'] = stability

                #Save dictionaries updated with stability scores
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 3 .
(5, 18) 0 0
Generating local Explanations for 206672_5
Run 0
Actual value  0
Run 1
Actual value  0
Run 2
Actual value  0
Stability: 0.75
Category 1 of 4 . Testing 2 of 3 .
(4, 18) 0 0
Generating local Explanations for 207735_4
Run 0
Actual value  0
Run 1
Actual value  0
Run 2
Actual value  0
Stability: 1.0
Category 1 of 4 . Testing 3 of 3 .
(5, 18) 0 0
Generating local Explanations for 212554_5
Run 0
Actual value  0
Run 1
Actual value  0
Run 2
Actual value  0
Stability: 0.78
Category 2 of 4 . Testing 1 of 3 .
(25, 18) 1 1
Generating local Explanations for 211212_25
Run 0
Actual value  1
Run 1
Actual value  1
Run 2
Actual value  1
Stability: 0.75
Category 2 of 4 . Testing 2 of 3 .
(22, 18) 1 1
Generating local Explanations for 213450_22
Run 0
Actual value  1
Run 1
Actual value  1
Run 2
Actual value  1
Stability: 0.86
Category 2 of 4 . Testing 3 of 3 .
(13, 18) 1 1
Generating local Explanations for 211850_13
Run 0
Actual value  1
Run 1
Actual v

In [39]:
if generate_shap:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #get the training data as a matrix
                trainingdata = feature_combiner.fit_transform(dt_train_bucket);
                #importance = generate_global_explanations(trainingdata,train_y, cls, feature_combiner)


                #create explainers now that can be passed later
                data_sample = shap.kmeans(trainingdata, #see "KMEANS EXPERIMENT" cell
                shap_explainer = shap.KernelExplainer(cls.predict, data_sample)
                tree_explainer = shap.TreeExplainer(cls)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    
                    for instance in category:
                        instance_no += 1    
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                
                        group = instance['input']

                        print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get Kernel SHAP explanations for instance
                        exp, rel_exp = create_samples(shap_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)

                        feat_pres = []

                        for iteration in rel_exp:
                            print("Kernel SHAP Iteration", rel_exp.index(iteration))
                            #print(iteration)

                            #Stability by index
                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        #by index
                                        presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['kernel_shap_stability'] = stability
                        
                        #Get Tree SHAP explanations for instance
                        exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)

                        feat_pres = []

                        for iteration in rel_exp:
                            print("Tree SHAP Iteration", rel_exp.index(iteration))
                            #print(iteration)

                            #Stability by index
                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        #by index
                                        presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['tree_shap_stability'] = stability

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1


Using 117965 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


Category 1 of 4 . Testing 1 of 3 .
(5, 18) 0 0
Generating local Explanations for 206672_5


MemoryError: Unable to allocate 272. GiB for an array with shape (2308, 15807310) and data type float64

In [84]:
#KMEANS EXPERIMENT
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sil_score = []
kmax = 100000
kmin = 20000
kvals = list(np.arange(kmin, kmax, 500))

for k in kvals:
    print ("testing value", kvals.index(k), "of", len(kvals))
    kmeans = KMeans(n_clusters = k).fit(trainingdata)
    labels = kmeans.labels_
    sil_score.append(silhouette_score(trainingdata, labels, metric = 'euclidean'))

opt = sil_score.index(max(sil_score))
opt_k = kvals[opt]

print(opt_k)

testing value 0 of 160
testing value 1 of 160
testing value 2 of 160


KeyboardInterrupt: 

In [90]:
cases = []
for each in dt_train_bucket['Case ID']:
    parts = str(each).split('_')
    case_id = parts[0]
    cases.append(case_id)

print (len(cases))
cases = list(dict.fromkeys(cases))
print(len(cases))

2040496
3748


In [ ]:
kmeans = KMeans(n_clusters = k).fit(trainingdata)
labels = kmeans.labels_
silhouette_score = silhouette_score(trainingdata, labels, metric = 'euclidean')

In [38]:
trainingdata.shape[0]*trainingdata.shape[1]

15807310

In [ ]:
tn = pd.DataFrame.from_records(tn_list)

In [ ]:
len(sample_instances)

In [ ]:
sample_size = 500


In [ ]:
tn = pd.DataFrame.from_records(tn_list)
tn

In [ ]:
tn_sample = pd.DataFrame(columns=tn.columns)
idxs = random.sample(range(0, tn.shape[0]-1), 500)
for i in idxs:
  sample = tn.loc[i]
  tn_sample = tn_sample.append(sample)
tn_sample

In [ ]:
x = tn_sample['caseID'].values
y1 = tn_sample['nr_events'].values
y2 = tn_sample['proba'].values

fig, ax1 = plt.subplots(figsize=(10, 10))

color = 'tab:red'
ax1.set_xlabel('No. Events')
ax1.set_ylabel('Prediction Probability', color=color)
ax1.plot(y1, y2, 'ro')
ax1.tick_params(axis='y', labelcolor=color)

#color = 'tab:blue'
#ax2 = ax1.twinx()
#ax2.set_ylabel('confidence', color=color)
#ax2.plot(x, y2, 'bo')
#ax2.tick_params(axis='y', labelcolor=color)

plt.show()

In [ ]:
if group.shape[0] in [5,10,25] and (count_d<9 or count_r<9):
  print(group.shape,test_y_group[0], pred)
  test_x_group= feature_combiner.fit_transform(group) 
  test_x=np.transpose(test_x_group[0])
  print('Generating local Explanations for', dataset_manager.get_case_ids(group))
  exp=generate_local_explanations(explainer, test_x, cls, test_y_group )
  if(test_y_group[0]==1):
      count_d=count_d+1
      exp_dlist.append(exp)
else:
    count_r=count_r+1
    exp_rlist.append(exp)


In [ ]:
tn['caseID'].unique